In [ ]:
# ===============================
# Improved XGBoost classifier for Sweet vs Tasteless
# ===============================

import pandas as pd
import numpy as np
import deepchem as dc
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, roc_curve
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import xgboost as xgb


data = pd.read_excel('../data/Rojas.xlsx')
df = data.dropna(subset=["SMILES", "class"]).copy()

# Convert labels: sweet = 1, tasteless = 0
df["label"] = df["class"].apply(lambda x: 1 if str(x).strip().lower() == "sweet" else 0)

# Morgan (Circular) fingerprint
featurizer = dc.feat.CircularFingerprint(size=1024)
X = featurizer.featurize(df["SMILES"])
y = df["label"].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# Calculate metrics
def calc_metrics_counts(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    Sn = tp / (tp + fn) if (tp + fn) > 0 else 0.0    # Sensitivity
    Sp = tn / (tn + fp) if (tn + fp) > 0 else 0.0    # Specificity
    NER = (Sn + Sp) / 2.0                            # Balanced accuracy
    return {"Sn": Sn, "Sp": Sp, "NER": NER, "TP": tp, "FP": fp, "TN": tn, "FN": fn}


# SMOTE + XGBoost
smote = SMOTE(random_state=42)
model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

pipe = Pipeline([
    ("smote", smote),
    ("xgb", model)
])

# Hyperparameter grid
param_grid = {
    "xgb__n_estimators": [100, 200, 300],
    "xgb__max_depth": [3, 4, 5],
    "xgb__learning_rate": [0.05, 0.1, 0.2],
    "xgb__subsample": [0.8, 0.9, 1.0],
    "xgb__colsample_bytree": [0.8, 0.9, 1.0],
}

# Grid search
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    scoring="balanced_accuracy",
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid.fit(X_train, y_train)

print("\n===== Best Parameters =====")
print(grid.best_params_)
print(f"Best CV balanced accuracy: {grid.best_score_:.3f}")

# Train best model
best_model = grid.best_estimator_
best_model.fit(X_train, y_train)

# ROC curve for threshold selection
y_proba = best_model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"\nOptimal threshold: {optimal_threshold:.2f}")

# Predictions using custom threshold
y_pred_train = (best_model.predict_proba(X_train)[:, 1] >= optimal_threshold).astype(int)
y_pred_test = (best_model.predict_proba(X_test)[:, 1] >= optimal_threshold).astype(int)

# Performance metrics
m_train = calc_metrics_counts(y_train, y_pred_train)
m_test = calc_metrics_counts(y_test, y_pred_test)

df_metrics = pd.DataFrame(
    {
        "Sn":   [m_train["Sn"], m_test["Sn"]],
        "Sp":   [m_train["Sp"], m_test["Sp"]],
        "NER":  [m_train["NER"], m_test["NER"]],
        "TP":   [m_train["TP"], m_test["TP"]],
        "FP":   [m_train["FP"], m_test["FP"]],
        "TN":   [m_train["TN"], m_test["TN"]],
        "FN":   [m_train["FN"], m_test["FN"]],
    },
    index=["Train", "Test"]
).round(2)

print("\n===== Train & Test Metrics =====")
print(df_metrics)

# Class distribution
print("\n===== Class Distribution =====")
print(df["label"].value_counts())


In [2]:
df_metrics = pd.DataFrame(
    {
        "Sn":   [m_train["Sn"], m_test["Sn"]],
        "Sp":   [m_train["Sp"], m_test["Sp"]],
        "NER":  [m_train["NER"], m_test["NER"]],
        "TP":   [m_train["TP"], m_test["TP"]],
        "FP":   [m_train["FP"], m_test["FP"]],
        "TN":   [m_train["TN"], m_test["TN"]],
        "FN":   [m_train["FN"], m_test["FN"]],
    },
    index=["Train", "Test"]
).round(2)

In [ ]:
print("\n===== Train & Test Metrics =====")
print(df_metrics)

# 12. Class distribution information
print("\n===== Class Distribution =====")
print(df['label'].value_counts())

In [6]:
# ===========================================
# XGBoost + MACCS model for Bitter vs Sweet
# Handles class imbalance with SMOTE
# Removes invalid SMILES automatically
# ===========================================

import pandas as pd
import numpy as np
import deepchem as dc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import xgboost as xgb
from rdkit import Chem

# 1️⃣ Load data
data = pd.read_excel('../data/Rojas.xlsx', sheet_name='Table 2S')
df = data.dropna(subset=["SMILES", "class"]).copy()

# 2️⃣ Label encoding: Bitter = 1, others = 0
df["label"] = df["class"].apply(lambda x: 1 if str(x).strip().lower() == "bitter" else 0)

# 3️⃣ Validate SMILES strings
def valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

# Identify invalid SMILES
invalid_smiles = df[~df["SMILES"].apply(valid_smiles)]
print("Invalid SMILES entries:")
print(invalid_smiles)

# Remove invalid SMILES
df = df[df["SMILES"].apply(valid_smiles)].reset_index(drop=True)
print(f"\nInvalid SMILES removed. Remaining sample size: {len(df)}")

# 4️⃣ MACCS keys fingerprint
featurizer = dc.feat.MACCSKeysFingerprint()
X = featurizer.featurize(df["SMILES"])
y = df["label"].values

# 5️⃣ Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6️⃣ Metric calculation
def calc_metrics_counts(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    Sn = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    Sp = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    NER = (Sn + Sp) / 2.0
    return {"Sn": Sn, "Sp": Sp, "NER": NER, "TP": tp, "FP": fp, "TN": tn, "FN": fn}

# 7️⃣ SMOTE + XGBoost pipeline
smote = SMOTE(random_state=42)
model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

pipe = Pipeline([
    ("smote", smote),
    ("xgb", model)
])

# 8️⃣ Hyperparameter grid search
param_grid = {
    "xgb__n_estimators": [100, 200, 300],
    "xgb__max_depth": [3, 4, 5],
    "xgb__learning_rate": [0.05, 0.1, 0.2],
    "xgb__subsample": [0.8, 0.9, 1.0],
    "xgb__colsample_bytree": [0.8, 0.9, 1.0],
}

grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    scoring="balanced_accuracy",
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid.fit(X_train, y_train)

print("\n===== Best Parameters =====")
print(grid.best_params_)
print(f"Best CV balanced accuracy: {grid.best_score_:.3f}")

# 9️⃣ Train the best model
best_model = grid.best_estimator_
best_model.fit(X_train, y_train)

# 🔟 Determine optimal threshold (ROC-based)
y_proba = best_model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"\nOptimal threshold: {optimal_threshold:.2f}")

# ⓫ Predict with optimal threshold
y_pred_train = (best_model.predict_proba(X_train)[:, 1] >= optimal_threshold).astype(int)
y_pred_test  = (best_model.predict_proba(X_test)[:, 1]  >= optimal_threshold).astype(int)

# ⓬ Performance metrics
m_train = calc_metrics_counts(y_train, y_pred_train)
m_test  = calc_metrics_counts(y_test,  y_pred_test)

df_metrics = pd.DataFrame(
    {
        "Sn":   [m_train["Sn"], m_test["Sn"]],
        "Sp":   [m_train["Sp"], m_test["Sp"]],
        "NER":  [m_train["NER"], m_test["NER"]],
        "TP":   [m_train["TP"], m_test["TP"]],
        "FP":   [m_train["FP"], m_test["FP"]],
        "TN":   [m_train["TN"], m_test{"TN"]],
        "FN":   [m_train["FN"], m_test["FN"]],
    },
    index=["Train", "Test"]
).round(2)

print("\n===== Train & Test Metrics =====")
print(df_metrics)

# ⓭ Class distribution
print("\n===== Class Distribution =====")
print(df["label"].value_counts())


[09:10:18] Explicit valence for atom # 2 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 2 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 3 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 5 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 6 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 7 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 9 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 11 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 3 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 3 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 3 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 2 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 4 N, 4, is greater than permitted
[09:10:18] Explicit valence for atom # 8 N, 4, is 

无效的 SMILES 条目：
    number                               name  \
274    275                Denatonium chloride   
275    276   Denatonium chloride derivative 1   
276    277   Denatonium chloride derivative 2   
277    278   Denatonium chloride derivative 3   
278    279   Denatonium chloride derivative 4   
279    280   Denatonium chloride derivative 5   
280    281   Denatonium chloride derivative 6   
281    282   Denatonium chloride derivative 7   
282    283   Denatonium chloride derivative 8   
283    284   Denatonium chloride derivative 9   
284   285^  Denatonium chloride derivative 10   
285    286  Denatonium chloride derivative 11   
286   287^  Denatonium chloride derivative 12   
287   288^  Denatonium chloride derivative 13   

                                                SMILES   class reference  \
274        CC[N](Cc1ccc(cc1)Cl)(CC(=O)Nc1c(C)cccc1C)CC  Bitter       NaN   
275                    ClC[N](CC(=O)Nc1c(C)cccc1C)(C)C  Bitter       NaN   
276                  

In [11]:
# ===========================================
# XGBoost + MACCS model (balanced dataset)
# Output: Sn / Sp / NER
# ===========================================

import pandas as pd
import numpy as np
import deepchem as dc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from rdkit import Chem

# 1️⃣ Load dataset (modify this path)
df = pd.read_csv("../data/Banerjee.csv")   # <-- change your filename here

# 2️⃣ Remove invalid rows
df = df.dropna(subset=["SMILES", "class"]).copy()

# Show class distribution
print("Class distribution:")
print(df["class"].value_counts())

# 3️⃣ Label encoding (automatic binary conversion)
classes = sorted(df["class"].unique())
mapping = {classes[0]: 0, classes[1]: 1}
print(f"\nLabel encoding: {mapping}")
df["label"] = df["class"].map(mapping)

# 4️⃣ Check SMILES validity
def valid_smiles(s):
    return Chem.MolFromSmiles(s) is not None

df = df[df["SMILES"].apply(valid_smiles)].reset_index(drop=True)
print(f"\nValid data count: {len(df)}")

# 5️⃣ MACCS fingerprint features
featurizer = dc.feat.MACCSKeysFingerprint()
X = featurizer.featurize(df["SMILES"])
y = df["label"].values

# 6️⃣ Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 7️⃣ Custom metrics
def calc_metrics_counts(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    Sn = tp / (tp + fn) if tp + fn > 0 else 0.0
    Sp = tn / (tn + fp) if tn + fp > 0 else 0.0
    NER = (Sn + Sp) / 2.0
    return Sn, Sp, NER, tp, fp, tn, fn

# 8️⃣ XGBoost hyperparameters
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 4, 5],
    "learning_rate": [0.05, 0.1, 0.2],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}

model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1
)

grid = GridSearchCV(
    model,
    param_grid=param_grid,
    scoring="balanced_accuracy",
    cv=5,
    n_jobs=-1,
    verbose=1
)
grid.fit(X_train, y_train)

print("\n===== Best Parameters =====")
print(grid.best_params_)

best_model = grid.best_estimator_

# 9️⃣ Predictions (threshold = 0.5)
y_pred_train = (best_model.predict_proba(X_train)[:, 1] >= 0.5).astype(int)
y_pred_test = (best_model.predict_proba(X_test)[:, 1] >= 0.5).astype(int)

# 🔟 Evaluate metrics
Sn_tr, Sp_tr, NER_tr, TP_tr, FP_tr, TN_tr, FN_tr = calc_metrics_counts(y_train, y_pred_train)
Sn_te, Sp_te, NER_te, TP_te, FP_te, TN_te, FN_te = calc_metrics_counts(y_test, y_pred_test)

df_metrics = pd.DataFrame({
    "Sn": [Sn_tr, Sn_te],
    "Sp": [Sp_tr, Sp_te],
    "NER": [NER_tr, NER_te],
    "TP": [TP_tr, TP_te],
    "FP": [FP_tr, FP_te],
    "TN": [TN_tr, TN_te],
    "FN": [FN_tr, FN_te],
}, index=["Train", "Test"]).round(2)

print("\n===== Train & Test Metrics =====")
print(df_metrics)


标签分布：
class
B    685
S    517
Name: count, dtype: int64

标签编码：{'B': 0, 'S': 1}

有效数据量：1202
Fitting 5 folds for each of 108 candidates, totalling 540 fits

===== 最优参数 =====
{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}

===== 训练集 & 测试集指标 =====
         Sn    Sp   NER   TP  FP   TN  FN
Train  0.99  0.99  0.99  408   5  543   5
Test   0.94  0.93  0.94   98   9  128   6
